# Markov Decison Processes

In Markov Decision Processes, one's actions affect the state of the world.  

Given a state $s$ and an action $a$, we know the probability it will end up in the next state $s'$ 

## Value Iteration/Backwards Induction/Dynamic Programming

Value iteration estimates the value of the best action at each state, $V^*(s)$, with recursion since

$$V^*(s) = \max_a\sum_{s'}P(s'|s,a)\left(R(s,a,s') + \gamma V^*(s')\right)$$

This equation is also known as the Bellman equation

Because of the discount factor, $\gamma$, our estimates will converge with the number of iterations since 

$$\mid V^*_i(s) - V^*_{i+1}(s) \mid  \leq \gamma^i\mid R_{max} - R_{min}\mid$$ 


## Gridworld Toy Example

Gridworld, a typical Reinforcement Learning problem, requires the agent to navigate to a goal loation and avoid possible pitfalls

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pdb

sns.set_context('poster')

%matplotlib inline

KeyboardInterrupt: 

### Let's imagine a 3 by 4 grid with 1 goal and 1 pitfall

In [3]:
grid = np.array([[-.01,-.01,-.01,1],[-.01,np.nan, -.01, -1],[-.01,-.01,-.01,-.01]])

In [4]:
sns.heatmap(grid, cmap='RdBu', annot=True, cbar = False, annot_kws={"size": 14})
plt.title('Reward for Each Grid Square')

NameError: name 'sns' is not defined

## In this world the agent can go up, down, left and right

### We'll assume 80% of the time the agent ends up going where it wants to go

### 20% of the time it veers off perpendicularly

### If it ends up in the pitfall or the goal, it's in the `exit` state and the game is over

In [ ]:
actions = ['up', 'down', 'left', 'right']
transition = pd.DataFrame(np.array([[.8, 0, .1, .1,], [0, .8, .1, .1], [.1, .1, .8, 0],
                                    [.1, .1, 0, .8]]), index = actions, columns = actions)

In [ ]:
sns.heatmap(transition, annot=True, cbar = False, annot_kws={"size": 14})
plt.title('Transition Probabilities for Each Action')

Right now our policy is unknown

In [ ]:
policy = pd.DataFrame([['unknown', 'unknown', 'unknown', 'exit'],
                       ['unknown', '', 'unknown', 'exit'],
                       ['unknown', 'unknown', 'unknown', 'unknown']])

In [ ]:
policy

## We'll now use Value Iteration to Solve for the Optimal Policy

Assume the discount factor is $0.75$

Let's set up our function to update the Q-values

In [ ]:
gamma = 0.75
row, col = np.shape(grid)
nactions = len(actions)

q_values = np.zeros((row,col,nactions))
v_values = np.copy(grid)

def index_grid(r, c, action):
    '''
    finds the right index for the next state based on the action name
    if the agent tries to go in the direction of a wall or edge,
    the agent says in the same place
    
    Args:
        r: row
        c: column
        action: action name
    
    Returns:
        r: updated row
        c: updated column
    '''
    if  action == 'up':
        if r > 0:
            r -= 1
    if action == 'down':
        if r < row - 1:
            r += 1
    if action == 'left':
        if c > 0:
            c -= 1
    if action == 'right':
        if c < col - 1:
            c += 1
    return r, c

def update_q(qvalues, vvalues):
    '''
    updates the value of each action given the current state and possible future states
    recall that the value of each action is the expected value of the current action 
    + future scenarios (see bellman equation above)
    
    Args:
        qvalues: array of qvalues for each state, action pair
        vvalues: array of vvalues for each state
    
    Returns:
        qvalues: updated qvalues
    '''
    for r in range(row):
        for c in range(col):
            #if we are in the exit squares, there is no where else to go so the value is fixed
            if abs(v_values[r, c]) == 1:
                qvalues[r, c, :] = v_values[r, c]
                continue
            #if we're in a nan square we can assume there's no value for that state
            elif np.isnan(grid[r, c]):
                qvalues[r, c, :] = np.nan
            for i, s in enumerate(actions):
                #we're going to loop over the actions twice because we're going from one state to another
                rewards = np.zeros(len(actions))
                for s_prime in range(len(rewards)):
                    #transitions with 0 probabiliy we can ignore
                    if transition[s].ix[actions[s_prime]] == 0:
                        continue
                    nrow, ncol = index_grid(r,c,s)
                    #attempts to go to the empty square result on the agent staying put
                    if np.isnan(grid[nrow, ncol]):
                        rewards[s_prime] = grid[r, c] + gamma*vvalues[r, c]
                    else:
                        rewards[s_prime] = grid[r, c] + gamma*vvalues[nrow, ncol]
                expected_value = np.dot(rewards, np.squeeze(transition[s]))
                qvalues[r,c,i] = expected_value
    return qvalues

## Now let's update the Q-values based on what we see on the grid

In [ ]:
qvalues_1 = update_q(q_values, v_values)
qvalues_1

## Once we know these Q-values we can update the V-values

Recall that 

$$V^*(s) = \max(Q^*(s,a))$$

and the optimal policy, $\pi^*(s)$, is

$$\pi^*(s) = \text{argmax}(Q^*(s,a))$$

In [ ]:
def update_v(qvalues, policy):
    '''
    updates the policy based on the qvalues
    recall that v*(s) = max(q*(s,a))
    also pi*(s) = argmax(q*(s,a))
    
    Args:
        qvalues: array of values for each state, action pair
        policy: array of actions for each state
    
    Returns:
        v_values_new: new array vvalues for each state
        policy: updated policy array
    '''
    v_values_new = np.max(qvalues, axis = 2)
    policy_indices = np.argmax(qvalues, axis = 2)
    for r in policy.index:
        for c in policy.columns:
            #we can ignore the blank states and terminal states
            if policy[c].ix[r] == 'exit' or policy[c].ix[r] == '':
                continue
            policy[c].ix[r] = actions[policy_indices[r, c]]
    return v_values_new, policy

In [ ]:
vvalues_1, policy_1 = update_v(qvalues_1, policy)
sns.heatmap(vvalues_1, cmap='RdBu', annot=True, cbar = False, annot_kws={"size": 14})
plt.title('V(s) After 1 Iteration')

In [ ]:
policy_1

# Now let's run this one more time to see how these values change again

In [ ]:
qvalues_2 = update_q(qvalues_1, vvalues_1)
qvalues_2

In [ ]:
vvalues_2, policy_2 = update_v(qvalues_2, policy_1)
sns.heatmap(vvalues_2, cmap='RdBu', annot=True, cbar = False, annot_kws={"size": 14})
plt.title('V(s) after 2 Iterations')

In [ ]:
policy_2

# We can now combine these two steps and run until convergence

In [ ]:
def value_iteration(qvalues, vvalues, policy, epsilon):
    qvalues = update_q(qvalues, vvalues)
    v_new, policy_new = update_v(qvalues, policy)  
    print(abs(v_new - vvalues))
    while (np.nanmax(abs(v_new - vvalues)) > epsilon):
        print('current epsilon: %0.5f' % np.nanmax(abs(v_new - vvalues)))
        vvalues = v_new
        qvalues = update_q(qvalues, vvalues)
        v_new, policy_new = update_v(qvalues, policy)
        
    return v_new, policy_new, qvalues

In [ ]:
v_final, policy_final, qvalues = value_iteration(qvalues_2, vvalues_2, policy_2, 0.001)

In [ ]:
sns.heatmap(v_final, cmap='RdBu', annot=True, cbar = False, annot_kws={"size": 14})
plt.title('Converged Values of V(s)')

In [ ]:
qvalues